# Setup Environment

In [ ]:
# To run locally, uncomment the below
# ! pip install -e ../

# To run in Onyx, uncomment the below
import sys
sys.path.append("..")

In [ ]:
from onyxgenai.extract import extract_text_from_folder, tokenize_sentences
from onyxgenai.embed import Embedder
from onyxgenai.generate import TextGenerator
from onyxgenai.model import ModelStore

# Extract text from mixed document types

In [ ]:
text,files = extract_text_from_folder("../data")
all_sentences = tokenize_sentences(text) 
sentences = [s for ss in all_sentences for s in ss] #flatten

# Embed and persist this data for RAG

In [ ]:
embedding_service = Embedder("http://embed.onyx-services/", model="all-MiniLM-L6-v2", model_version=8, num_workers=1, collection_name="test_collection")
embeddings= embedding_service.embed_text(sentences)


# Start/Serve the Embedding Model

In [ ]:
model_service = ModelStore("http://store.onyx-services/")
app_name = model_service.serve_model(model="all-MiniLM-L6-v2", model_version=8, app_name="minilm")
print(app_name)

# Embed Prompt for Vector Search

In [ ]:
data = ["This is a test sentence.", "Another test sentence for prediction."]
embeddings = model_service.predict_text(app_name=app_name, data=data)
print(embeddings)

# Perform Vector Search

In [ ]:
## TODO: Update to call onyx service when available
from qdrant_client import QdrantClient

client = QdrantClient("http://qdrant.qdrant")
search_result = client.search(
    collection_name="test_collection", query_vector=embeddings[0], limit=3
)

print(search_result)

# Execute a RAG workflow

In [ ]:
prompt = "What ticketing tools has DHS utilized for IT services?"
answer = generator_service.generate(prompt,context=embedding_service.query_text(prompt) )


In [ ]:
print(answer)

# Shutdown the services

In [ ]:
model_service.stop_model(app_name)


# Get List of Available Models

In [ ]:
model_service = ModelStore("http://store.onyx-services/")
models = model_service.get_models()
print(models)